
Anti-Aliasing Functions in Interferometry
=========================================

In [ ]:
%matplotlib inline

import sys
sys.path.append('../..')

from matplotlib import pylab

pylab.rcParams['figure.figsize'] = 12, 10

import numpy
import scipy
import scipy.special

from crocodile.clean import *
from crocodile.synthesis import *
from crocodile.simulate import *
from crocodile.antialias import *

from util.visualize import *


## Test setup

We will use a field of view of 0.004 radian. We place one
source within the field of view ($l=m=0.001$) and another 5 times stronger source just outside ($l=m=0.0025$).

In [ ]:
vlas = numpy.genfromtxt("../../data/configurations/VLA_A_hor_xyz.csv", delimiter=",")
uvw = xyz_to_baselines(vlas, numpy.arange(0,numpy.pi,0.04), numpy.pi/4) / 5

yyone = simulate_point(uvw, 0.001, 0.001)
yytwo = yyone + 5*simulate_point(uvw, 0.0025, 0.0025)

## Simple Imaging


Imaging without convolution with just the first source within field of view:

In [ ]:
theta = 0.004
lam = 30000
d,_,_=do_imaging(theta, lam, uvw, None, yyone, simple_imaging)
show_image(d, "simple[yyone]", theta)
print(d[40:60,40:60].std())

If we now again do simple imaging with both sources, we see that the strong
source at (0.0025, 0.0025) is getting "aliased" back into the field of view at (-0.0015, -0.0015):

In [ ]:
d,_,_=do_imaging(theta, lam, uvw, None, yytwo, simple_imaging)
show_image(d, "simple[yytwo]", theta)
print(d[40:60,40:60].std())

## Anti-aliasing function

This is an example anti-aliasing function to use. It is separable, so we can work equivalently with one- or two-dimensional representations:

In [ ]:
support = 6
aa = anti_aliasing_function(int(theta*lam), 0, support)
aa2 = numpy.outer(aa, aa)
pylab.rcParams['figure.figsize'] = 7, 5
pylab.plot(theta*coordinates(int(theta*lam)), aa); pylab.show()
show_image(aa2, "aa2", theta)

After FFT-ing and extracting the middle this is what the oversampled anti-aliasing
kernel looks like in grid space:

In [ ]:
oversample = 128
r = numpy.arange(-oversample*(support//2), oversample*((support+1)//2)) / oversample
kv=kernel_oversample(aa, oversample, support)
pylab.plot(r, numpy.transpose(kv).flatten().real);

Imaginary part is close to nil:

In [ ]:
pylab.plot(r, numpy.transpose(kv)[::-1].flatten().imag);

## Gridding with anti-aliasing function

This is the image of single source within field of view *without correcting the taper*. Note that brightness falls off
towards the edges of the picture. This is because applying the anti-aliasing convolution kernel is equivalent to multiplying the picture with the anti-aliasing function shown above.

In [ ]:
d,_,_=do_imaging(theta, lam, uvw, None, yyone, conv_imaging, kv=kv)
pylab.rcParams['figure.figsize'] = 12, 10
show_image(d, "aa_{one}", theta)
print(d[40:60,40:60].std())

However, as the anti-aliasing function never goes to zero, we can easily revert this effect by dividing out the anti-aliasing function:

In [ ]:
show_image(d/numpy.outer(aa, aa), "aa'_{one}", theta)
print((d/aa2)[40:60,40:60].std())

Now we have restored image performance with just a single source in the field of view. In fact,
imaging is a good deal cleaner than before (and the source slightly stronger), as with
oversampling we are now taking fractional coordinates of visibilities into account.

Bust most critically if we now add back the source outside of the field of view, it gets
suppressed strongly. Because of its strength we still see noise centered around its off-screen 
position at (0.0025, 0.0025), but the source itself is gone:

In [ ]:
d,_,_=do_imaging(theta, lam, uvw, None, yytwo, conv_imaging, kv=kv)
show_image(d/numpy.outer(aa, aa), "aa'_{two}", theta)
print((d/aa2)[40:60,40:60].std())